In [11]:
from folium.plugins import TimeSliderChoropleth
import folium
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from branca.colormap import linear
from folium.plugins import TimeSliderChoropleth


assert 'naturalearth_lowres' in gpd.datasets.available
datapath = gpd.datasets.get_path('naturalearth_lowres')
gdf = gpd.read_file(datapath)

print(gdf.columns)
print()
print(gdf.shape)

# how many countries do we have? 
print(gdf["name"].unique().shape)

In [25]:
# how many time sample points do we want?
n_periods = 48

n_sample = 40

assert n_sample < n_periods

datetime_index = pd.date_range('2016-1-1', periods=n_periods, freq='M')
dt_index_epochs = datetime_index.astype(int) // 10**9
dt_index = dt_index_epochs.astype('U10')

# for i in zip(datetime_index, dt_index_epochs, dt_index):
#     print(i)

In [58]:

styledata = {}

for country in gdf.index:
    df = pd.DataFrame(
        {'color': np.random.normal(size=n_periods),
         'opacity': np.random.normal(size=n_periods)},
        index=dt_index
    )
    # Take cumulative sum to pretend as though each point is built on the other
    # df = df.cumsum()
    # Randomly shuffle these points now
    # df.sample(n_sample, replace=False).sort_index()
    styledata[country] = df

In [ ]:
# Normalize this data to get the colour map information
max_color, min_color, max_opacity, min_opacity = 0, 0, 0, 0

for country, data in styledata.items():
    max_color = max(max_color, data['color'].max())
    min_color = min(max_color, data['color'].min())
    max_opacity = max(max_color, data['opacity'].max())
    max_opacity = min(max_color, data['opacity'].max())



cmap = linear.PuRd_09.scale(min_color, max_color)

def norm(x):
    return (x - x.min()) / (x.max() - x.min())


In [75]:
# Now change the data in the style data for each country
# for country, data in styledata.items():
#     data['color'] = data['color'].apply(cmap)
#     data['opacity'] = norm(data['opacity'])


In [76]:

# print(styledata[0].head())


In [77]:
# Finally, make the style dictionary to be in the form folium needs
# mainkey_in_datadf : { timestamp_in_epoch : {"Color":HEX_COLOR, "opacity":float(0,1) } } }
styledict = {
    str(country): data.to_dict(orient='index') for
    country, data in styledata.items()
}
# for k,v in styledict['0'].items():
#     print(k, "-->\n", v)

In [ ]:

m = folium.Map(zoom_start=2)

g = TimeSliderChoropleth(
    gdf.to_json(),
    styledict=styledict,

).add_to(m)
 
# Banana hammock
import webbrowser
filepath = os.path.join(os.getcwd(), 'a.html')
m.save(filepath)
webbrowser.open('file://' + filepath)
